In [60]:
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline

In [61]:
import hopsworks
import pandas as pd

In [62]:
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/195733
Connected. Call `.close()` to terminate connection gracefully.


In [63]:
archive_df = pd.read_csv("./data/98230-1-archive.csv", delimiter=';', skiprows=9)
latest_df = pd.read_csv("./data/98230-1-latest.csv", delimiter=';', skiprows=9)
temp_df = pd.concat([archive_df, latest_df])
temp_df

,Datum,Tid (UTC),Lufttemperatur,Kvalitet,Unnamed: 4,Tidsutsnitt:
0,1996-10-01,00:00:00,10.2,G,NaN,Kvalitetskontrollerade historiska data (utom d...
1,1996-10-01,01:00:00,9.7,Y,NaN,Tidsperiod (fr.o.m.) = 1996-10-01 00:00:00 (UTC)
2,1996-10-01,03:00:00,9.9,G,NaN,Tidsperiod (t.o.m.) = 2023-10-01 06:00:00 (UTC)
3,1996-10-01,04:00:00,9.9,Y,NaN,Samplingstid = Ej angivet
4,1996-10-01,05:00:00,9.9,Y,NaN,Kvalitetskoderna:
...,...,...,...,...,...,...
3123,2024-01-02,11:00:00,-5.5,G,NaN,NaN
3124,2024-01-02,12:00:00,-5.4,G,NaN,NaN
3125,2024-01-02,13:00:00,-5.6,G,NaN,NaN
3126,2024-01-02,14:00:00,-5.9,G,NaN,NaN


In [64]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 238383 entries, 0 to 3127
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Datum           238383 non-null  object 
 1   Tid (UTC)       238383 non-null  object 
 2   Lufttemperatur  238383 non-null  float64
 3   Kvalitet        238383 non-null  object 
 4   Unnamed: 4      0 non-null       float64
 5   Tidsutsnitt:    26 non-null      object 
dtypes: float64(2), object(4)
memory usage: 12.7+ MB


In [65]:
temp_df.describe()

,Lufttemperatur,Unnamed: 4
count,238383.000000,0.0
mean,7.977728,NaN
std,8.110398,NaN
min,-20.700000,NaN
25%,1.800000,NaN
50%,7.400000,NaN
75%,14.300000,NaN
max,34.000000,NaN


### Cleaning

In [66]:
temp_df = temp_df.drop(columns=["Unnamed: 4", "Tidsutsnitt:"])
temp_df

,Datum,Tid (UTC),Lufttemperatur,Kvalitet
0,1996-10-01,00:00:00,10.2,G
1,1996-10-01,01:00:00,9.7,Y
2,1996-10-01,03:00:00,9.9,G
3,1996-10-01,04:00:00,9.9,Y
4,1996-10-01,05:00:00,9.9,Y
...,...,...,...,...
3123,2024-01-02,11:00:00,-5.5,G
3124,2024-01-02,12:00:00,-5.4,G
3125,2024-01-02,13:00:00,-5.6,G
3126,2024-01-02,14:00:00,-5.9,G


In [67]:
temp_df["Kvalitet"].value_counts()

Kvalitet
G    158171
Y     80212
Name: count, dtype: int64

In [68]:
temp_df["Date-Time"] = pd.to_datetime(temp_df["Datum"] + " " + temp_df["Tid (UTC)"])
temp_df = temp_df.drop(columns=["Datum", "Tid (UTC)"])
temp_df

,Lufttemperatur,Kvalitet,Date-Time
0,10.2,G,1996-10-01 00:00:00
1,9.7,Y,1996-10-01 01:00:00
2,9.9,G,1996-10-01 03:00:00
3,9.9,Y,1996-10-01 04:00:00
4,9.9,Y,1996-10-01 05:00:00
...,...,...,...
3123,-5.5,G,2024-01-02 11:00:00
3124,-5.4,G,2024-01-02 12:00:00
3125,-5.6,G,2024-01-02 13:00:00
3126,-5.9,G,2024-01-02 14:00:00


In [70]:
temp_df.sort_values("Date-Time")
temp_df.drop_duplicates()

,Lufttemperatur,Kvalitet,Date-Time
0,10.2,G,1996-10-01 00:00:00
1,9.7,Y,1996-10-01 01:00:00
2,9.9,G,1996-10-01 03:00:00
3,9.9,Y,1996-10-01 04:00:00
4,9.9,Y,1996-10-01 05:00:00
...,...,...,...
3123,-5.5,G,2024-01-02 11:00:00
3124,-5.4,G,2024-01-02 12:00:00
3125,-5.6,G,2024-01-02 13:00:00
3126,-5.9,G,2024-01-02 14:00:00


In [71]:
new_names = {}
for name in temp_df.columns:
    new_names[name] = name.lower().replace('-', '_')
temp_df = temp_df.rename(columns=new_names)
temp_df.columns

Index(['lufttemperatur', 'kvalitet', 'date_time'], dtype='object')

In [72]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 238383 entries, 0 to 3127
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   lufttemperatur  238383 non-null  float64       
 1   kvalitet        238383 non-null  object        
 2   date_time       238383 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 7.3+ MB


### Insert our Wine DataFrame into a FeatureGroup

In [73]:
temp_fg = fs.get_or_create_feature_group(
    name="weather",
    version=1,
    primary_key=["date_time"],
    description="Weather dataset")
temp_fg.insert(temp_df)

Uploading Dataframe: 100.00% |██████████| Rows 238383/238383 | Elapsed Time: 00:30 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/195733/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fc60dbdb6a0>, None)